<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/Data/action_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import re
class Data:
  def __init__(self, file_path):
    self.dataFrame = pd.read_csv(file_path, encoding='cp949')
  
  def get_content_by_col(self, colomn):
    # 같은 내용의 카테고리 합치기
    self.dataFrame[colomn] = self.dataFrame[colomn].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)
    content = list(set(self.dataFrame[colomn].values.tolist()))
    content = sorted(content)
    return content

### Install library

In [4]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install customized_konlpy # 형태소 사용자 사전 추가
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

키워드 추출기

In [5]:
!pip3 install krwordrank # 키워드 추출기
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize


def keyword_tracker_KR(text_group): 
  texts = [normalize(text, english=True, number=True) for text in text_group]  # 문장내 특수문자 제거하기
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )
  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10
  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)
  keyword = []
  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
      keyword.append(word)
  return keyword






#형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt

okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls

def keyword_tracker_Okt(text_group): # memo-문장이 들어있는 리스트
  text_group = [ re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',text) for text in text_group]  # 문장내 특수문자 제거하기
  text_group = ' '.join(text_group)
  text_group=morph_and_stopword(text_group) # 형태소 분리
  text_group=Counter(text_group)
  text_group_rank=text_group.most_common(5) # 상위 5개
  return text_group_rank


In [6]:
# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

## 업무구조 - 4단계

In [14]:
from numpy import nan

class Main_divider:
  def __init__(self):

    self.factor_trigger = re.compile(r'고장[\s]*부위|문의[\s]*내용|증상[\s]*상세|문의[\s]*사항|문의[\s]*구분')
    self.factor_check = re.compile(r'확인[\s]*[내용|처리[\s]*내용]*|확인[\s]*사항') 
    self.factor_figure = re.compile(r'안내[\s]*[약속[\s]*내용|처리[\s]*사항|처리[\s]*내용|사항]|안내사항') 
    self.factor_promise = re.compile(r'약속[\s]*[내용|사항]') 
    # action부분에서 이러한 단어가 있으면 중요한 문장으로 생각한다.
    self.actor = 'ABS|ADAS|AUX|DPF|ESC|GPS|ODO|OTP|RPM|SD|TCS|USB|가리개|가스|각도|강제|개방|거치대|게이지|경고등|'\
    '경보|경적|계기판|고장|고정|공기압|공조|교체|교환|구리스|구토|그리스|글로브|급발진|기름|기어|기억|깜빡|꺼|꼽|나사|'\
    '내비게이션|내장제|냄새|네트[\s]*워크|녹이|누유|단말기|단차|닫|담배|담뱃재|동물 털|동파|드럼|들리|등받이|디스크|따뜻|떨리|떨림|떨어|'\
    '라우터|라이트|레버|레이더|마감|마모|마스터키|만차|매트|모드|미러|미점등|밀림|바디|바람[\s]*빠|바퀴|박스|받침대|발판|방전|방지|방향|배선|'\
    '배출|배치|배터리|버튼|범퍼|변속|보관함|보닛|보조|보충|볼륨|부족|부착|부팅|분실|불량|불빛|브레이크|블랙박스|블루투스|비치함|'\
    '뽑|사고조사|사이드|삽입|선루프|선바이저|선팅|세차|센서|소거|소리|소음|손상|손잡이|수동|스마트[\s]*키|스캐[너|서]|스크래치|스티커|스패너|습기|'\
    '시거잭|시동|시야|시트|신호|실내등|실행|쏠|악취|안나|안내[\s]*[약속[\s]*내용|사항]|안전벨트|알림|언|얼라이먼트|얼어|업그레이드|'\
    '업데이트|에어|에이필러|엑셀|엔 와 향|엔진|연결|연동|연료|열|열선|오디오|오류|오일|옥스 트라|온도|와이파이|와이퍼|외관|요소수|운전석|워셔액|워시|'\
    '원격|위생|위치|유리|유심|음성|이동|인식|인터넷|입차|잠금|장착|장치|재시작|전구|전원|전조등|점등|점프|제동등|제어|조수석|조절|좌석|주유|주행등|'\
    '중계기|진단|찢|차 세 제어|차가운|찬|창문|처리[\s]*[내용|사항]|청소|체크|초기화|출차|충돌|충전|카드|카메라|캡|커넥터|컨트롤|컴파운드|코드절상|클리닝|'\
    '키 아웃|키 인식|타는|타이어|탈거|탐색|태깅|테스트|테이프|통신|트렁크|티씨에스|파손|파스|판단|패드|펑크|편의|포켓|포트|품목|퓨즈|플레이트|하이패스|핸들|'\
    '혼유|화면|환경이슈|후미등|후방|후진|훼손|휠|히터'
    self.actor_figure = re.compile(self.actor) 
    self.service_info ='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*|'\
    '(관제\s예약\s)*URL[주소:\s]*[^\n]+|'\
    '(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*|'\
    '([원]*예약번호|블락번호|고객운행)[^\n]*|'\
    '((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*|'\
    '([\\n]*(채팅[\s]*상담|톡|요청자)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인/요청자 제거



  def chunck_maker(self, memo):
    memo = re.sub(r'#|_', ' ', memo.upper())
    memo = re.sub(r'\([^\)\()]*\)|([0-9]+/[0-9]+)', '', memo) # 삭제: 괄호 내용 & 날짜 
    memo = re.sub(r'[ㄴ|ㅁ|ㅡ|\-|(\t)|:|\|]', '\n', memo) # 문장 구획하기
    memo = re.sub(r'( )*(\n+)( )*', '\n', memo)
    memo = re.sub(r'(\n)+', '\n', memo)
    memo = re.sub(r'( )+', ' ', memo)
    return memo
    


  def track_Trigger(self, memo):
    # 1....... 주어진 문장을 \n로 분할하여 리스트만들기  
    memo = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\(\)\s]', '', memo)
    memo = re.sub(self.service_info, '', memo)
    memo = self.chunck_maker(memo)
    memo_chunck = memo.split('\n')

    # 2............   필요한 리스트 초기화
    flag = 'None'
    figure=[]
    check=[]
    trigger=[]
    semi_trigger = True

    # 3.................... 쪼개진 문장에서 Trigger / 확인사항/ figure / 나누기 
    for i in memo_chunck:
        if self.factor_trigger.search(i):
          flag = 'trigger_mode'
          continue
        elif self.factor_check.search(i): 
          flag = 'check_mode'
          continue
        elif self.factor_figure.search(i): 
          flag = 'figure_mode'
          continue
        elif self.factor_promise.search(i): 
          flag = 'promise_mode'   

        if flag == 'trigger_mode': trigger.append(i)
        elif flag == 'check_mode': check.append(i)
        elif flag == 'figure_mode':
          if self.actor_figure.search(i): 
            if re.compile('진행|확인|안내|가능|요청|발송|불가|적용|해결|문자|조정|수긍|과실|양해|는지|부탁|도움|도와|부과').search(i): semi_trigger = False
            if semi_trigger: 
              trigger.append(i)
              print(i)
            else: 
              figure.append(i)
        elif flag == 'promise_mode':  pass

    # 4........ 결과 출력하기
    print('▷ Chunk: ', memo_chunck)
    if len(trigger) != 0: print("증상상세:", trigger)
    if len(check)  != 0: print("확인내용:", check)
    if len(figure) != 0: print("안내처리:", figure)
    return figure, check, trigger 





  def refine_action(self, action_list):
    action_list = [x for x in action_list if x is not nan]
    action_list = "\n".join(action_list)
    action_list = re.sub(r'[네|내]비[게이션]*', '내비게이션', action_list)
    action_list = re.sub(r'본넷|본네트|본닛|보닛트', '보닛', action_list)
    action_list = re.sub(r'라이트/전구', '라이트 및 전구', action_list)
    action_list = re.sub(r'문제어|도어( )*제어', '문 제어', action_list)
    action_list = re.sub(r'삽입', '장착', action_list)
    action_list = re.sub(r'교환', '교체', action_list)
    action_list = re.sub(r'긴출', '긴급출동', action_list)
    action_list = re.sub(r'블박', '블랙박스', action_list)
    action_list = re.sub(r'엔[.|,]*와[.|,]*향', '엔와향', action_list)
    action_list = re.sub(r'(좌[ /.,]*우)|(전[ /.,]후)|(A/S)|(\d[\s]*본|\d[\s]*짝|\d[\s]*EA)', '', action_list)
    # 전 좌우 타이어 -> 타이어
    action_list = re.sub(r'(전|앞|뒤|좌|우|전륜|후륜|하향|상향|하부|상부|내부|외부|좌측|우측| )+ *({})'.format(self.actor), r' \2', action_list) # 후지움
    action_list = self.chunck_maker(action_list)
    action_list = re.split('\n|/|\.|,', action_list) # 문장분리

    real_action_list = []
    no_use = re.compile(r'->|[0-9]{3,10}|@|가온|담당자') 
    for i in action_list:
      i = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',i)
      try: i = correction(i.strip()) # 맞춤법 교정하기 2 (네이버 검사기)
      except: pass
      i = re.sub(r'( )+등', r'등', i)
      if self.actor_figure.search(i):
        if not no_use.search(i): #완벽하게 통과된 
          real_action_list.append(i)
    return real_action_list

In [15]:
# 상담내역 파일 load & 객체 생성
file_path = '/content/drive/MyDrive/2-2._lesion_info_table.csv'
data = Data(file_path)
inspect_type = data.get_content_by_col("lesion_detail_type") # 존재하는 카테고리 종류 모음
print(inspect_type)

['ADAS_고장', 'ADAS_파손', 'USB포트_고장', '거치대_분실', '거치대_파손', '경고등_ABS', '경고등_DPF(배출가스)', '경고등_ESC(차세제어)', '경고등_배터리', '경고등_브레이크', '경고등_스패너(서비스알림)', '경고등_에어백', '경고등_엔진오일', '경고등_엔진체크', '경고등_연료부족', '경고등_온도게이지', '경고등_요소수부족', '경고등_전/후측방 충돌 경고', '경고등_통합경고등(느낌표)', '글로브박스_파손', '내비게이션_고장', '내비게이션_분실', '내비게이션_업데이트', '내비게이션_인터넷연결불가', '내비게이션_파손', '내장제_파손', '단말기_배선', '단말기_신호불량', '단말기_탈거', '단말기_파손', '도어_고장', '도어_소음', '도어_파손', '라이트/전구_경고등(고장)', '라이트/전구_파손', '룸미러_파손', '목받침대_고장/분실', '문제어불가_옥스트라', '문제어불가_카드태깅', '발판매트_분실', '번호판_분실', '번호판_파손', '본넷_고장', '본넷_파손', '브레이크_고장', '브레이크_밀림', '브레이크_소음', '블랙박스_고장', '블랙박스_분실', '블랙박스_탈거', '블랙박스_파손', '사고조사_요청', '사이드미러_고장', '사이드미러_파손', '사이드브레이크_해제불가', '선바이저_파손', '시거잭_고장', '시동_고장', '시동_꺼짐', '시동_배터리방전', '시동_키인식불가(키아웃)', '시동_혼유', '시트_시트조절_고장', '시트_열선시트_고장', '시트_파손', '실내등_고장', '안심번호_분실', '안심번호_연동오류', '안전벨트_고장', '에어컨_고장', '에어컨_소음', '에어컨_악취', '오디오_AUX', '오디오_고장', '오디오_버튼파손/분실', '오디오_블루투스', '오디오_소음', '와이퍼_고장', '와이퍼_교체필요', '와이퍼_파손', '외관_누유', '외관_파손(상부)', '외관_파손(하부)', '외부스티커_훼손', '워셔액_

In [16]:
divider = Main_divider()

In [ ]:
# Trigger & Action 데이터는 파일로 저장하기
f_trigger = open('/content/drive/MyDrive/Trigger_list_final.txt','w')
f_action = open("/content/drive/MyDrive/action_list_final.csv", "w")
f_action.write( 'inspect_type' + ',' + 'content'+ '\n')


In [ ]:

dic_trigger={}
dic_check={}
dic_figure={}
dic_action={}
inspect_num=0
for inspect in inspect_type: # 특정 카테고리
  all_case_trigger = []
  all_case_check = []
  all_case_figure = []
  all_case_action = []

  id_list = data.dataFrame.groupby('lesion_detail_type').get_group(inspect) 
  id_list = set(id_list['lesion_id'].values.tolist())
  for id in id_list: # 카테고리 내의 사건번호
    case = data.dataFrame.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    case = case.drop_duplicates(['memo']) # memo의 중복행 제거
    content = case['memo'].values.tolist()
    print("============================================\n{}▶ Origin: ".format(inspect_type[inspect_num]), content)

    # 다음 원소에도 Trigger가 존재하는가 
    point = 0
    for i in range(0, len(content)):
        if divider.factor_trigger.search(str(content[i])):
          figure, check, trigger = divider.track_Trigger(content[i]) # 로직 분석기 돌리기
          all_case_trigger.append(trigger)
          all_case_check.append(check)
          all_case_figure.append(figure)
          point = i
    else: 
          action = content[point+1:]
          action = divider.refine_action(action) # Action 중에서 필요한 정보만 추출하기
          print('차후수리: ', action)
          all_case_action += action


  dic_trigger[inspect_num]= all_case_trigger
  dic_check[inspect_num]= all_case_check
  dic_figure[inspect_num]= all_case_figure
  dic_action[inspect_num]= all_case_action


  # Trigger & Action 데이터 저장
  for i in sorted(all_case_action): # Action 데이터 저장
    f_action.write(inspect_type[inspect_num] + ',' + i + '\n')
  for i in sorted(all_case_trigger): # trigger 데이터 저장
    f_trigger.write(str(inspect_num)+'\t'+ i +'\t'+str(inspect_num)+'\n') 
    
  print('\n\n*****************************************************************************************************************')
  print(" >> ", inspect_type[inspect_num])
  print("증상상세:", all_case_trigger)
  print("확인내용:", all_case_check)
  print("안내처리:", all_case_figure)
  print("차후수리:", all_case_action)
  print('*****************************************************************************************************************\n\n')


  inspect_num+=1



## Action 데이터 세부 정제

In [62]:
file_path = '/content/drive/MyDrive/action_list_final.csv' # 앞서 저장했던 Action데이터 호출
action_data = pd.read_csv(file_path, names=['inspect_type', 'content'])

In [63]:
action_data.head()

,inspect_type,content
0,inspect_type,content
1,ADAS_고장,ADAS 단말기 오작동 이상 없고
2,ADAS_고장,ADAS 연결 정상 문 제어
3,ADAS_고장,ADAS 점점 이상 없음
4,ADAS_고장,ADAS 조치 완료


In [54]:

Trigger_fact = '[으]*로$|불가|불량|인입|오류|밀림|고장|이상|장애|탈락|방전|안됨|분실|미인식|파손|필요|없|않|안( )+|안( )*되|부족' #이상없음 주의
Action_fact = '재결합|결합|강제|개방|고정|공조|교체|도포|리셋|배출|밸런스|배치|방청|보충|변경|분사|부착|분리|부팅|삽입|소거|설치|소진|세팅|업그레이드|업데이트|재연결|제거|연결|이동|원격|원복|인식|잠금|장착|점검|작업|재시작|점프|주입|접촉|제어|조절|조립|정리|조정|진단|청소|충전|초기화|클리닝|터치업|코드절상|키 아웃|키 인식|탐색|태깅|판단|회수|환기'
divider = {'Trigger':Trigger_fact, 'Action_divide':'\.| 이후 |후 | 이*후에도* |수리 *완료| *완료| 결과 |부탁[^\s]*|요청[^\s]*|하여[^\s]*', 'Action_fact':Action_fact}

# f_C = open('/content/drive/MyDrive/action_list2.txt','w')
# f = open("/content/drive/MyDrive/action_list2.csv", "w")
# f.write( 'inspect_type' + ',' + 'content'+ '\n')

for inspect in inspect_type: 
  try:
   case = action_data.groupby('inspect_type').get_group(inspect) 
   actions = sorted(list(set(case['content'].values.tolist())))
  except: pass
  print('\n\n*****************************************************************************************************************')
  print(inspect,"\n*****************************************************************************************************************")
  origin_action = []
  action = []
  new_action = []
  trigger_sec = []

  # 하나의 문장도 쪼갤 수 있으면 쪼개기
  for i in actions: 
    i = re.sub(r'니다', '니다.', i) 
    i = re.sub(r' 테스트', '.테스트', i) 
        #print(i)
    if re.compile(divider['Action_divide']).search(i):
        #print(re.split(divider['Action_divide'], i))
        new_action += re.split(divider['Action_divide'], i)
    else: 
        new_action.append(i)


  # 여기서부터 검사하기
  for i in list(set(new_action)):
    if re.compile(divider['Trigger']).search(i):
      #print("문제:", i)
      trigger_sec.append(re.split(divider['Trigger'], i)[0])
    elif re.compile(divider['Action_fact']).search(i):
      rei = i.strip()
      if '및' in i:  # '및' 뒤의 내용이 결과인 경우는 뒷부분 버리기    
        if rei[0] == '및': rei = rei[1:]
        if not re.compile(divider['Action_fact']).search(" ".join(i.split('및')[1:])): 
          print(rei)
          rei = i.split('및')[0]
          #print(rei)
      if '정상' in rei or "예정" in rei: 
        pass
      else: 
        if len(rei.split(" ")) > 1: print(rei)
      # if re.compile('(으)*로( )*방문( )*예정|완료|부탁|매칭|요청|진행|확인|(및 )*정상').search(i): 
      #  rei = re.sub(r'((으)*로( )*방문( )*예정|완료|부탁|매칭|요청|진행|확인|(및 )*정상)+[^\s]*', '', rei)
      #if rei != i: print("액션:", i, "//", rei)     
      #else: print("액션:", i)
    else:
      pass#print("그밖:", i)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
운전석 라이트 전구 교체
조수석 방향지시등 전구 교체
리어 백 패널 판금 고정
운전석 앞 소켓 교체 주행등 전구 교체
타이어 공기압 경고등 소거
조수석 헤드램프 전구 교체
엔 와 향 운전석 라이트 전구 교체
소프트웨어 업그레이드
부품 수급 및 주간 주행등 전구 소켓 교체
보조 제동등 교체
SD카드 회수 및 발송
SD카드 회수 
후미등 교체
운전석 제동등 전구 교체
조수석 브레이크 전 구 교체
제동등 교체 경고등 해제
엔 와 향 교체
주간 주행등 조수석 교체
후우 차폭등 전구 교체
W 업그레이드
운전석 브레이크 전구 교체
운전석 방향지시등 전구 및 소켓 교체
브레이크 보조 제동등 교체
전조등 교체
운전석 미등 전구 교체
우측 헤드 램프 전구 교체
경고등 소거
충동 방지 보조 시스템 점검 문구 점등 범퍼 눈 치우니까 경고등 소거됨
전구 교체
배터리 교체
브레이크 전구 교체
좌측 주간 주행등 교체
전우 하향등 교체
운전석 주간 주행등 전구 교체
주유카드 1월 11일 교체 매칭 확인
운 주행등 소켓 주행등 전구 교체
주간 주행등 점검
전좌 하향등 교체
엔 와 향 조수석 턴 시그널 전구 교체
코일 세트 교체
TCU 업그레이드
라이트 전구 양쪽 교체
조수석 브레이크등 전구 교체
스로틀 보디 탈거 클리닝 점화플러그 교체 부동액 완충
조 뒤 넘어 판 전구 조 라이트 전구 교체했습니다
폭스바겐 서초 서비스센터 오일 보충
브레이크등 전구 양쪽 교체
공기압 보충
계기판 리셋
운전석 브레이크등 전구 교체
조수석 라이트 전구 교체
좌측 소켓 주간 주행등 교체


*****************************************************************************************************************
글로브박스_파손 
*******************************************************************************

# 문장을 유사도로 그룹화해서 대표문장 추출하기 (정확도 낮음) 

In [ ]:
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np
from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.add_dictionary('얼라인먼트|브레이크실린더', 'Noun')


# 문장 데이터 집어넣기
only_content = action
print(len(action))
origin_x_data = np.array(only_content)
origin_x_data_T = np.array(only_content)

for i, document in enumerate(origin_x_data):
      print('전:', document)
      nouns = twitter.morphs(origin_x_data[i]) # 형태소로 분리하기 (트위터_원본)
      origin_x_data_T[i] = ' '.join(nouns) 
      print('후:', origin_x_data_T[i])
      # nouns = morph_and_stopword(origin_x_data[i])
      # origin_x_data_T[i] = ' '.join(nouns) 
      # print('후:', origin_x_data_T[i])


origin_x_data_T = np.array(list(origin_x_data_T))

In [ ]:
def make_correct_sentence(list_sentence, keyword_list):
  min_text = list_sentence[0]
  for i in list_sentence:
    for key in keyword_list:
      if key not in i:
        break
    else:
      if len(i) < len(min_text):
        min_text = i
  print('best:', min_text)
  return min_text

In [ ]:

correct_sentence = [] # 추출된 correct sentence

def check_simillity(sentence_list):
  #print(sentence_list) # 담긴 문장 확인하기 
  vect = TfidfVectorizer()
  x_data = vect.fit_transform(sentence_list)
  sentence_list = sentence_list.tolist()
  cosine_similarity_matrix = (x_data * x_data.T)
  print('size: ', cosine_similarity_matrix.shape) 

  index_list = [] # 지워야할 정보
  cnt = 0
  for i in range(len(sentence_list)):
    if i not in index_list: # 이미 어떤 문장의 유사도에 포함되어있으면 검사하지말기
      cnt += 1
      sentences_for_keyword = [] # 유사한 문장담기
      vals = cosine_similarity(x_data[i], x_data) # 타깃하는 문장의 유사도 데이터 
      idx = vals.argsort()[0] # 오름차순에 해당하는 인덱스 정보를 담음 
      flat = vals.flatten() # 유사도 오름차순 정렬
      flat.sort()
      for index in range(len(flat)-1, -1, -1):
        if flat[index] < 0.9: # 유사한 정도
          break
        else:
            real_index = sentence_list.index(sentence_list[idx[index]]) # 실제 문장 리스트에서 몇번째 인덱스인지 flat은 유사도 오름차순된거라 실제 문장의 인덱스가 아님
            index_list.append(real_index) 
            sentences_for_keyword.append(sentence_list[real_index])
            
      if len(sentences_for_keyword) > 2: # 유사한 문장 그룹내의 키워드 추출     
        print('------------------------\n')
        for i in sentences_for_keyword:
          print(i)
        print(' \n▶ 「KRWordRank」형태소 분석:')
        try:
          okt_keyword = keyword_tracker_KR(sentences_for_keyword)
          print(okt_keyword)
        except: pass
        if len(okt_keyword) > 1: # 키워드가 하나이상이고
          main_sentence  = make_correct_sentence(sentences_for_keyword, okt_keyword)
          correct_sentence.append(main_sentence)
      else:
        correct_sentence.append(sentences_for_keyword[0])
  print(cnt)
  for i in set(correct_sentence):
    print(i)
  return 
check_simillity(origin_x_data)
print('\n\n*****************************************************************************************************************')
check_simillity(origin_x_data_T)